# ETF Top Performer Analyzer - POC

In [1]:
# Import Modules
import pandas as pd
import os
import json
import requests
from dotenv import load_dotenv
import matplotlib.pyplot as plt
import alpaca_trade_api as tradeapi
from pathlib import Path
import sqlalchemy as sql
import ETFHistoryDownload as hist
from datetime import date
import logging
from dateutil.relativedelta import relativedelta


## STEP1. IMPORT ETF AND CONSTITUENTS (CSV FILES) AND INSERT INTO SQL Tables

In [2]:
#CSV Data Path
etf_list_path = Path('./Resources/etf_list.csv')
etf_constituents_path = Path('./Resources/etf_holdings.csv')

In [3]:
#1.Load ETF List from CSV
etf_list_df = pd.read_csv(etf_list_path,index_col='etf', parse_dates=True, infer_datetime_format=True)

,category,type
etf,,
RYT,Sector,Technology
XSW,Sector,Technology
XLK,Sector,Technology
USRT,Sector,Real Estate
XLRE,Sector,Real Estate
RWR,Sector,Real Estate
BBUS,Purpose,Large_blend
JMOM,Purpose,Large_growth
SPMD,Purpose,Mid_blend


In [4]:
#2.Load ETF Constituents from CSV
etf_constituents_df = pd.read_csv(etf_constituents_path, parse_dates=True, infer_datetime_format=True)
etf_constituents_df

,etf,symbol,name,pct_holding
0,RYT,ENPH,Enphase Energy Inc,0.0192
1,RYT,ANET,Arista Networks Inc,0.0153
2,RYT,CRM,Salesforce.com Inc,0.0153
3,RYT,XLNX,Xilinx Inc,0.0152
4,RYT,PAYC,Paycom Software Inc,0.0151
...,...,...,...,...
1073,XLK,CTXS,Citrix Systems Inc,0.0011
1074,XLK,JNPR,Juniper Networks,0.0009
1075,XLK,DXC,Dxc Technology Company,0.0007
1076,XLK,WU,Western Union Company,0.0007


In [ ]:
# Create DB
# Database connection string
eft_data_connection_string = 'sqlite:///./Resources/etf.db'
# Database engine
etf_data_engine = sql.create_engine(eft_data_connection_string, echo=True)
# Create two tables from the dataframe
etf_list_df.to_sql('ETF_LIST', etf_data_engine, index_label='etf', if_exists='replace')
etf_constituents_df.to_sql('ETF_CONSTITUENTS', etf_data_engine, index=False, if_exists='replace')

In [6]:
# Select all symbols and names from the Finance sector
sql_query = """
SELECT distinct symbol as name
FROM ETF_CONSTITUENTS
UNION
SELECT distinct etf
FROM ETF_LIST
"""

In [11]:
etf_symbols = pd.read_sql_query(sql_query, eft_data_connection_string)
etf_symbols

,name
0,A
1,AAON
2,AAPL
3,AAT
4,ABBV
...,...
632,ZS
633,ZTS
634,ZUO
635,QQQ


In [17]:
#DEBUG: If need to add more tickers, then add here
etf_symbols = pd.DataFrame(['QQQ','SPY','GLD'],columns = ["name"]) 
etf_symbols

,name
0,QQQ
1,SPY
2,GLD


# STEP2. Download historical Data into SQL DB

In [ ]:
#Tweak - Due to API limitation - only download new names by cossing check with existing data

sql_query = """
SELECT distinct symbol as name
FROM STOCK_HISTORY
"""
etf_symbols_exist = pd.read_sql_query(sql_query, eft_data_connection_string)

remaining_list = pd.merge(etf_symbols, etf_symbols_exist, how='outer', indicator=True)
remaining_list = remaining_list.loc[remaining_list._merge == 'left_only', ['name']]

# If need to reset the table (Please don't activate - will take long time to re-download)
#hist.drop_table('STOCK_HISTORY')
if len(remaining_list) > 0:
    day_t0 = date.today()
    hist.download_EFT_holdings(remaining_list, day_t0)


## STEP3. Analyze Stock Performance
Analyze the historical data and store the data with index: symbol

Owner: Albert
Symbol | Total Return | Sharpe Ration | PER  | Corelation |..... * 660 symbols
moving average
Return
Sharpe Ratio
P/E ratio

In [31]:
#hist.drop_table('STOCK_HISTORY')
historical_px_matrix = hist.get_price_history_by_period(day_t)
historical_px_matrix 

period,D0,D7_W1,M1,M3,M6,Y0_YTD,Y1,Y2,Y3
symbol,,,,,,,,,
A,159.81,157.050,152.49,131.88,131.88,118.66,112.63,76.63,67.75
AAON,76.71,72.420,68.99,66.04,66.04,64.70,63.54,50.13,43.84
AAPL,150.83,148.990,143.01,126.86,126.86,129.41,116.32,260.14,208.49
AAT,39.62,38.540,38.22,34.95,34.95,27.41,25.13,46.74,39.05
ABBV,115.60,114.820,111.19,116.19,116.19,105.41,95.18,85.22,87.73
...,...,...,...,...,...,...,...,...,...
ZIXI,8.50,8.170,7.46,6.64,6.64,8.36,7.07,6.96,6.75
ZM,258.40,278.700,255.24,288.49,288.49,359.78,413.20,69.84,NaN
ZS,352.63,318.815,266.50,165.77,165.77,196.21,128.77,43.89,41.71


In [47]:
#1. download historical data 1 ~ 2 symbolsCorrelations...
#2. Build Calculation Lib to caclculate TR, SR, PER, CORR, ....
#3. Create the matrix with result
# Symbol | Moving average | Total Return | Sharpe Ratio | PER  | Corelation |..... * 660 symbols
#4. Store back to database table - "PERFORMANCE_MATRIX"

def stock_analzer(etf_symbol):
    alpaca = tradeapi.REST(
    alpaca_api_key,
    alpaca_secret_key,
    api_version="v2")
    return 

In [32]:
preformance_matrix = hist.get_performance_by_period((day_t), True)
preformance_matrix

period,D0,D7_W1,M1,M3,M6,Y0_YTD,Y1,Y2,Y3,D7_W1%,M1%,M3%,M6%,Y0_YTD%,Y1%,Y2%,Y3%
symbol,,,,,,,,,,,,,,,,,
A,159.81,157.85,150.01,131.88,131.88,118.66,112.63,76.85,67.29,1.241685,6.532898,21.178344,21.178344,34.678915,41.889372,107.950553,137.494427
AAON,76.71,71.69,67.91,66.04,66.04,64.70,63.54,50.24,43.26,7.002371,12.958327,16.156875,16.156875,18.562597,20.727101,52.687102,77.323162
AAPL,150.83,150.00,142.80,126.86,126.86,129.41,116.32,262.20,204.47,0.553333,5.623249,18.894845,18.894845,16.552044,29.668157,-42.475210,-26.233677
AAT,39.62,38.14,39.01,34.95,34.95,27.41,25.13,46.91,39.39,3.880440,1.563702,13.361946,13.361946,44.545786,57.660167,-15.540397,0.583905
ABBV,115.60,116.53,110.34,116.19,116.19,105.41,95.18,85.67,88.78,-0.798078,4.767084,-0.507789,-0.507789,9.667015,21.454087,34.936384,30.209507
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZIXI,8.50,8.21,7.39,6.64,6.64,8.36,7.07,7.11,6.67,3.532278,15.020298,28.012048,28.012048,1.674641,20.226308,19.549930,27.436282
ZM,258.40,280.50,253.86,288.49,288.49,359.78,413.20,68.38,NaN,-7.878788,1.788387,-10.430171,-10.430171,-28.178331,-37.463698,277.888271,NaN
ZS,352.63,326.01,267.55,165.77,165.77,196.21,128.77,44.03,40.69,8.165394,31.799664,112.722447,112.722447,79.720707,173.844840,700.885760,766.625707


In [45]:
#TO-DO: Albert - 
# Could you add analysis columns to performance_matrix?
# you can pull history from STOCK_HISTORY table
day_1 = date.today() + relativedelta(days=-1)
year_1 = day_1 + relativedelta(years=-1)

sql_query = f"""
SELECT date, symbol, close FROM STOCK_HISTORY WHERE date > '{year_1}'
"""

stock_history = pd.read_sql_query(sql_query, eft_data_connection_string)
stock_history


,date,symbol,close
0,2020-11-10,A,110.20
1,2020-11-11,A,109.44
2,2020-11-12,A,109.48
3,2020-11-13,A,109.98
4,2020-11-16,A,109.53
...,...,...,...
160771,2021-11-03,GLD,165.78
160772,2021-11-04,GLD,167.66
160773,2021-11-05,GLD,169.82
160774,2021-11-08,GLD,170.45


# 4. Cherry Picking to create our portfolio
pick best based on performance scoring matrix per investment criteria and benchmark EFTs
Owner: Ken Lee

In [ ]:
def Stock_picker(eft_symbol, criteria):
    #stock_picker.py
    #code here
    #return dataframe with pick
    return

In [ ]:
sector_choice = int(input("Enter Sector Choice: (1. Technology 2. RealEstate 3.Both):"))
strategy_choice = int(input("Enter Strategy Preference: (1.LargeCap 2. MidCap 3. SmallCap 4.All Blended):"))
    
sector_technology = 0
sector_realestate = 0
cap_large = 0
cap_mid = 0
cap_small = 0

if sector_choice == 1:
    sector_technology = 1
elif sector_choice == 2:
    sector_realestate = 1
else:
    sector_technology = 1
    sector_realestate = 1


if strategy_choice == 1:
    sector_technology = 1
elif sector_choice == 2:
    sector_realestate = 1
else:
    sector_technology = 1
    sector_realestate = 1

    

In [46]:
#TO-DO: Minglu Li
#  Could you add monteCarlo simularation on 3 names: SPY, QQQ, GLD?
# GLD is the placeholder of our bespoke portfolio
day_1 = date.today() + relativedelta(days=-1)
year_1 = day_1 + relativedelta(years=-1)

sql_query = f"""
SELECT date, symbol, close FROM STOCK_HISTORY WHERE date > '{year_1}' and symbol in ('SPY','QQQ','GLD')
"""

stock_3ETF = pd.read_sql_query(sql_query, eft_data_connection_string)
stock_3ETF

,date,symbol,close
0,2020-11-10,QQQ,283.46
1,2020-11-11,QQQ,289.84
2,2020-11-12,QQQ,288.42
3,2020-11-13,QQQ,291.48
4,2020-11-16,QQQ,293.05
...,...,...,...
751,2021-11-03,GLD,165.78
752,2021-11-04,GLD,167.66
753,2021-11-05,GLD,169.82
754,2021-11-08,GLD,170.45


# 5. Performance Forecast Visualization - Validation
Show our portfolio performance with Chart and Matrix
Run Montecarlo simulation
Compare performance Forecast
Owner: Minglu

In [ ]:
def performance_forecast():
    from MCForecastTools import MCSimulation
    # performance_forecast including benchmarks
    # code here
    # See 05-Inst_Simulation_of_Stock_Price_Trajectory/
    return


# 6. REPORT
1. Summarize the analysis and results
2. Future improvement and limitation --no Dividend and CF analysis

In [ ]:
def report_visualization():
    # Import the hvPlot library
    import hvplot.pandas
    #code here
    # See 06-Data-Visualization-with-PyViz/
    # Report with charts
    return